In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
citiesFile = "output/cities.csv"

In [3]:
citiesSummary = pd.read_csv(citiesFile)
citiesSummary.head()

,Unnamed: 0,City,Lat,Lng,Temp Max,Humidity,Cloudness,windSpeed,country,date
0,0,San Patricio,28.0170,-97.5169,98.17,41,33,16.46,US,1658863946
1,1,Kanniyākumāri,8.0833,77.5667,77.95,83,28,8.23,IN,1658863999
2,2,Saskylakh,71.9167,114.0833,66.27,85,28,4.21,RU,1658863883
3,3,Punta Arenas,-53.1500,-70.9167,39.22,70,75,13.80,CL,1658863946
4,4,Hilo,19.7297,-155.0900,83.28,92,100,3.44,US,1658863865


In [7]:
gmaps.configure(api_key=g_key)

In [8]:
# Store 'Lat' and 'Lng' into  locations 
locations = citiesSummary[["Lat", "Lng"]].astype(float)
Weight = citiesSummary["Humidity"].astype(float)

In [9]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
tempFilter = citiesSummary.loc[(citiesSummary['Temp Max'] > 70) & (citiesSummary["Temp Max"] < 80) & 
                               (citiesSummary['windSpeed'] < 10) & (citiesSummary['Cloudness'] == 0)]
tempFilter.head()

,Unnamed: 0,City,Lat,Lng,Temp Max,Humidity,Cloudness,windSpeed,country,date
33,33,Gōtsuchō,35.0000,132.2167,75.63,86,0,4.16,JP,1658864007
64,64,Fort-Shevchenko,44.5086,50.2630,77.56,61,0,7.14,KZ,1658864014
72,72,Grand Gaube,-20.0064,57.6608,70.20,76,0,1.99,MU,1658864016
227,227,Erice,38.0370,12.5865,75.56,79,0,8.05,IT,1658864056
229,229,Mogadouro,41.3403,-6.7119,75.13,33,0,9.31,PT,1658864057


In [17]:
tempFilter['Hotel Name']= ""


tempFilter

/var/folders/2w/fy2bjdfs7nz5_675nfk08_jh0000gp/T/ipykernel_61918/2330774380.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempFilter['Hotel Name']= ""


,Unnamed: 0,City,Lat,Lng,Temp Max,Humidity,Cloudness,windSpeed,country,date,Hotel Name
33,33,Gōtsuchō,35.0000,132.2167,75.63,86,0,4.16,JP,1658864007,
64,64,Fort-Shevchenko,44.5086,50.2630,77.56,61,0,7.14,KZ,1658864014,
72,72,Grand Gaube,-20.0064,57.6608,70.20,76,0,1.99,MU,1658864016,
227,227,Erice,38.0370,12.5865,75.56,79,0,8.05,IT,1658864056,
229,229,Mogadouro,41.3403,-6.7119,75.13,33,0,9.31,PT,1658864057,
246,246,Shetpe,44.1667,52.1167,77.88,43,0,4.72,KZ,1658864061,
253,253,Ābyek,36.0667,50.5500,73.00,48,0,3.51,IR,1658864064,
273,273,Jarinu,-23.1014,-46.7283,79.18,39,0,4.47,BR,1658863959,
304,304,Chiang Rai,19.9167,99.9167,78.78,94,0,1.01,TH,1658864076,
400,400,Wajima,37.4000,136.9000,75.69,87,0,3.80,JP,1658864102,


In [30]:

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in tempFilter.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        tempFilter.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [31]:
tempFilter

,Unnamed: 0,City,Lat,Lng,Temp Max,Humidity,Cloudness,windSpeed,country,date,Hotel Name
33,33,Gōtsuchō,35.0000,132.2167,75.63,86,0,4.16,JP,1658864007,Super Hotel Gotsu Ekimae
64,64,Fort-Shevchenko,44.5086,50.2630,77.56,61,0,7.14,KZ,1658864014,Dostyk Hotel
72,72,Grand Gaube,-20.0064,57.6608,70.20,76,0,1.99,MU,1658864016,Veranda Paul et Virginie Hotel & Spa
227,227,Erice,38.0370,12.5865,75.56,79,0,8.05,IT,1658864056,PARCO DEGLI AROMI RESORT e SPA
229,229,Mogadouro,41.3403,-6.7119,75.13,33,0,9.31,PT,1658864057,Hotel Trindade Coelho
246,246,Shetpe,44.1667,52.1167,77.88,43,0,4.72,KZ,1658864061,Gostinitsa Shetpe
253,253,Ābyek,36.0667,50.5500,73.00,48,0,3.51,IR,1658864064,زادگاه
273,273,Jarinu,-23.1014,-46.7283,79.18,39,0,4.47,BR,1658863959,Hotel & Restaurante Fazenda D'anape
304,304,Chiang Rai,19.9167,99.9167,78.78,94,0,1.01,TH,1658864076,Imsuk Homestay at Chiang Rai
400,400,Wajima,37.4000,136.9000,75.69,87,0,3.80,JP,1658864102,Hotel Route-Inn Wajima


In [29]:
marker_locations = tempFilter[['Lat', 'Lng']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))